<a href="https://colab.research.google.com/github/Tamilarasee/Pyspark/blob/main/Tutorial%206-Example%20Of%20Pyspark%20ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Examples Of Pyspark ML

In [7]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [8]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [9]:
training.show()



+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [10]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [11]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [12]:
  #[Age,Experience]----> new feature--->independent feature

In [13]:
# we create x and y -->independent and dependt feature for splitting and training
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [14]:
output=featureassembler.transform(training)

In [15]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [26]:
output.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Independent Features: vector (nullable = true)



In [17]:
finalized_data=output.select("Independent Features","Salary")

In [18]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [20]:
### Coefficients (weights of the features)
regressor.coefficients

DenseVector([28.4757, 1271.3568])

In [21]:
### Intercepts (bias - the default value when feature weights are zero)
regressor.intercept

14299.83249581293

In [22]:
### Prediction
pred_results=regressor.evaluate(test_data)

#evaluate ----- This method returns a RegressionMetrics object, which contains multiple evaluation results

In [23]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|         [31.0,10.0]| 30000|27896.147403685143|
+--------------------+------+------------------+



In [24]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(2103.852596314857, 4426195.747020763)